# Intruder Detection System using OpenCV and MOG2

Real-time intruder detection using background subtraction. Detected motion is highlighted in red with an on-screen warning.

In [31]:
import cv2
import numpy as np

In [32]:
# Configuration
KERNEL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (19, 19)) 
#Change size if the detection is too sensitive/frigid

# Initialize
cap = cv2.VideoCapture(0) #if camera isnt detected change this
#create the subtractor object
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False) 

print("Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply MOG2 background subtraction
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    fg_mask = bg_subtractor.apply(blurred)
    
    # Clean mask: threshold to remove shadows, then open and close to connect the mask
    _, thresh = cv2.threshold(fg_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, KERNEL)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, KERNEL)

    # Find contours
    contours, _ = cv2.findContours(cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    intruders = []
    for contour in contours:
        if cv2.contourArea(contour) > 5000:
            intruders.append(contour)
    
    # Draw red highlights on intruders
    if intruders:
        overlay = frame.copy()
        cv2.drawContours(overlay, intruders, -1, (0, 0, 255), -1)
        frame = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)
        
        #draw every contour
        for c in intruders:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        # Draw warning box (top-right corner)
        fh, fw = frame.shape[:2]
        cv2.rectangle(frame, (fw - 200, 10), (fw - 10, 60), (0, 0, 180), -1)
        cv2.rectangle(frame, (fw - 200, 10), (fw - 10, 60), (0, 0, 255), 2)
        cv2.putText(frame, "! WARNING !", (fw - 190, 32), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, "INTRUDER", (fw - 175, 52), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Display
    cv2.imshow('Intruder Detection', frame)
    cv2.imshow('Mask', cleaned)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Press 'q' to quit
